In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install pytorch-metric-learning
!pip install faiss-gpu
!unzip "/content/drive/MyDrive/ADL Work/PA1/matching_task.zip"

In [ ]:
from pytorch_metric_learning import losses, miners, distances, reducers, testers
from pytorch_metric_learning.utils.accuracy_calculator import AccuracyCalculator
from torchvision import datasets
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms, models
import numpy as np
import pandas as pd
import os
from skimage import io, transform
from PIL import Image
from sklearn.utils import shuffle
import copy

In [ ]:
class MatchingDataset(Dataset):
    """Matching dataset."""

    def __init__(self, df, root_dir, transform=None):
        """
        Args:
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.df = df
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        folder, img_name = self.df.iloc[idx]["image_id"].split("_")
        img_path = os.path.join(self.root_dir, folder, img_name)
        image = Image.open(img_path)
        label = self.df.iloc[idx, 1]
        label = label-1

        if self.transform:
            image = self.transform(image)
        return image, label

In [ ]:
train_df = pd.read_csv("/content/matching_task/matching_train.csv")
train_df = shuffle(train_df, random_state=1411)

image_ids = train_df['image_id'].unique()

valid_ids = image_ids[-486:]
train_ids = image_ids[:-486]

valid_df = train_df[train_df['image_id'].isin(valid_ids)]
train_df = train_df[train_df['image_id'].isin(train_ids)]

valid_df.shape, train_df.shape

((486, 2), (1944, 2))

In [ ]:
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((110,300)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((110,300)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [ ]:
def train(model, loss_func, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, labels) in enumerate(train_loader):
        data, labels = data.to(device), labels.to(device)
        optimizer.zero_grad()
        embeddings = model(data)
        loss = loss_func(embeddings, labels)
        loss.backward()
        optimizer.step()
        if batch_idx % 20 == 0:
            print("Epoch {} Iteration {}: Loss = {}".format(epoch, batch_idx, loss))

In [ ]:
def get_all_embeddings(dataset, model):
    tester = testers.BaseTester()
    return tester.get_all_embeddings(dataset, model)

In [ ]:
def test(train_set, test_set, model, accuracy_calculator):
    train_embeddings, train_labels = get_all_embeddings(train_set, model)
    test_embeddings, test_labels = get_all_embeddings(test_set, model)
    train_labels = train_labels.squeeze(1)
    test_labels = test_labels.squeeze(1)
    print("Computing accuracy")
    accuracies = accuracy_calculator.get_accuracy(test_embeddings, 
                                                train_embeddings,
                                                test_labels,
                                                train_labels,
                                                False)
    print("Test set accuracy (Precision@1) = {}".format(accuracies["precision_at_1"]))
    return accuracies

In [ ]:
device = torch.device("cuda")

train_dataset = MatchingDataset(train_df, "/content/matching_task/train", data_transforms["train"])
test_dataset = MatchingDataset(valid_df, "/content/matching_task/train", data_transforms["val"])

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=128)

model = models.resnet50(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 512)
model.to(device)

loss_func = losses.ArcFaceLoss(243, 512, margin=28.6, scale=64).to(device)

optimizer = optim.Adam(list(model.parameters())+ list(loss_func.parameters()), lr=0.001)
num_epochs = 30

In [ ]:
accuracy_calculator = AccuracyCalculator(include = ("precision_at_1",), k = 1)
best_acc = 0
for epoch in range(1, num_epochs+1):
    train(model, loss_func, device, train_loader, optimizer, epoch)
    acc = test(train_dataset, test_dataset, model, accuracy_calculator)
    if best_acc <= acc["precision_at_1"]:
        best_acc = acc["precision_at_1"]
        best_model = copy.deepcopy(model.state_dict())

In [ ]:
torch.save(best_model, "/content/drive/MyDrive/ADL Work/matching_model_30epochs.pth")

In [ ]:
best_acc

0.9218106995884774

In [ ]:
cos = nn.CosineSimilarity(dim=0, eps=1e-6)
test_iter = iter(test_loader)
model.eval()

In [ ]:
sample, labels = next(test_iter)
sample = sample.to(device)
out = model(sample)

In [ ]:
np.unique(labels.numpy(), return_counts=True)

(array([  0,  10,  11,  17,  24,  27,  28,  30,  31,  33,  36,  38,  40,
         41,  45,  49,  50,  51,  52,  54,  55,  59,  60,  61,  63,  66,
         67,  68,  71,  74,  76,  80,  81,  83,  84,  85,  86,  98,  99,
        103, 109, 110, 111, 112, 113, 114, 115, 118, 119, 121, 125, 129,
        131, 141, 142, 143, 144, 145, 148, 149, 152, 155, 157, 158, 161,
        166, 171, 174, 177, 179, 180, 182, 187, 189, 192, 193, 197, 198,
        201, 203, 204, 205, 206, 207, 208, 212, 213, 214, 215, 216, 218,
        219, 220, 222, 224, 225, 226, 229, 235, 238, 239, 240, 242]),
 array([1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 2, 3, 1,
        1, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 3, 1, 3, 2, 2, 1, 2, 1, 1,
        1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 2, 1, 1, 2, 1, 1,
        1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]))

In [ ]:
(labels == 21).nonzero(as_tuple=True)

(tensor([22, 88]),)

In [ ]:
cos(out[22], out[88])

tensor(0.9858, device='cuda:0', grad_fn=<DivBackward0>)